In [ ]:
from dotenv import load_dotenv

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(model="deepseek-r1-distill-llama-70b")

In [ ]:
llm.invoke("hi")

In [8]:
import requests
from pydantic import BaseModel
from typing import Literal
from dotenv import load_dotenv
import os
import json

load_dotenv()

class Flight(BaseModel):
    departure_airport_code: str
    arrival_airport_code: str
    departure_date: str
    cabin_class: Literal["Economy", "Business", "First"]
    number_of_adults: int
    number_of_children: int

class FlightChecker:
    def __init__(self):
        self.api_key = os.getenv("FLIGHT_API_KEY")
        self.url = "https://api.flightapi.io/onewaytrip"
        
    def check_flight_availability(self, flight: Flight):
        response = requests.get(
            f"{self.url}/{self.api_key}/"
            f"{flight.departure_airport_code}/"
            f"{flight.arrival_airport_code}/"
            f"{flight.departure_date}/"
            f"{flight.number_of_adults}/"
            f"{flight.number_of_children}/0/"
            f"{flight.cabin_class}/USD"
        )

        if response.status_code != 200:
            return {"error": f"API request failed with status {response.status_code}"}

        data = response.json()  # ✅ convert Response object into dict
        flights_data = []

        # Optional carriers mapping
        carriers_map = {}
        if "carriers" in data:
            for carrier in data["carriers"]:
                carriers_map[carrier["id"]] = carrier.get("name", "")

        # Extract itineraries
        for itinerary in data.get("itineraries", []):
            cheapest_price = itinerary.get("cheapest_price", {}).get("amount", None)

            for option in itinerary.get("pricing_options", []):
                for item in option.get("items", []):
                    url = item.get("url", "")
                    if "flight|" in url:
                        parts = url.split("flight|")[1:]
                        for p in parts:
                            flight_info = p.split("|")
                            if len(flight_info) >= 7:
                                carrier_id = flight_info[0]
                                flight_number = flight_info[1]
                                origin = flight_info[2]
                                departure = flight_info[3]
                                destination = flight_info[4]
                                arrival = flight_info[5]

                                flights_data.append({
                                    "price": cheapest_price,
                                    "carrier_id": carrier_id,
                                    "carrier_name": carriers_map.get(int(carrier_id), "Unknown"),
                                    "flight_number": flight_number,
                                    "origin": origin,
                                    "departure": departure,
                                    "destination": destination,
                                    "arrival": arrival
                                })
                                

        if not flights_data:
            return []
        
        # Sort by price and return top 5
        sorted_flights = sorted(flights_data, key=lambda x: x["price"] if x["price"] else float('inf'))
        return sorted_flights[:5]


        return minimum_price[:5]


checker = FlightChecker()

print(checker.check_flight_availability(
    Flight(
        departure_airport_code="DEL",
        arrival_airport_code="BOM",
        departure_date="2025-10-17",
        cabin_class="Economy",
        number_of_adults=1,
        number_of_children=0
    )
))


[{'price': 57.36, 'carrier_id': '-32213', 'carrier_name': 'IndiGo', 'flight_number': '6814', 'origin': '10957', 'departure': '2025-10-17T07:00', 'destination': '10075', 'arrival': '2025-10-17T09:15'}, {'price': 57.36, 'carrier_id': '-32213', 'carrier_name': 'IndiGo', 'flight_number': '6814', 'origin': '10957', 'departure': '2025-10-17T07:00', 'destination': '10075', 'arrival': '2025-10-17T09:15'}, {'price': 57.36, 'carrier_id': '-32213', 'carrier_name': 'IndiGo', 'flight_number': '6814', 'origin': '10957', 'departure': '2025-10-17T07:00', 'destination': '10075', 'arrival': '2025-10-17T09:15'}, {'price': 57.36, 'carrier_id': '-32213', 'carrier_name': 'IndiGo', 'flight_number': '6814', 'origin': '10957', 'departure': '2025-10-17T07:00', 'destination': '10075', 'arrival': '2025-10-17T09:15'}, {'price': 57.36, 'carrier_id': '-32213', 'carrier_name': 'IndiGo', 'flight_number': '6814', 'origin': '10957', 'departure': '2025-10-17T07:00', 'destination': '10075', 'arrival': '2025-10-17T09:15'}]